In [3]:
!pip install iisignature

  Using cached iisignature-0.24-cp310-cp310-linux_x86_64.whl

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import time
import tqdm
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

In [11]:
from torchkan.data import timeseries
from torchkan.models import TKAT, TKAN

In [3]:
num_hidden = 100
num_heads = 4
num_embedding = 19
n_ahead = 30
sequence_length = 5 * n_ahead
sequence_length

150

In [4]:
def get_crypto_dataloaders(
    path = "torchkan/data/data.parquet", 
    n_ahead = 30
):
    # Load California housing dataset
    df = timeseries.load_crypto(path)
    
    X_scaler, X_train, X_test, \
        X_train_unscaled, X_test_unscaled, \
            y_scaler, y_train, y_test, \
                y_train_unscaled, y_test_unscaled, \
                    y_scaler_train, y_scaler_test = timeseries.generate(df, sequence_length, n_ahead)

    # Create data loaders (optional, if you want to batch and shuffle the data)
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=1, shuffle=False)

    return train_loader, test_loader

In [5]:
train_loader, test_loader = get_crypto_dataloaders()
[t.shape for t in train_loader.dataset.tensors], [t.shape for t in train_loader.dataset.tensors]

/home/miles/Development/notebooks/torchkan/torchkan/data/timeseries.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(data, dtype=dtype).to(device)


([torch.Size([20607, 150, 19]), torch.Size([20607, 30])],
 [torch.Size([20607, 150, 19]), torch.Size([20607, 30])])

In [6]:
shape = train_loader.dataset.tensors[0].shape
torch.split(train_loader.dataset.tensors[0], (1,) * shape[-1], -1)[0].shape

torch.Size([20607, 150, 1])

In [13]:
model = TKAN(input_size=sequence_length, hidden_size=num_hidden)
model

TKAN(
  (tkan_cells): ModuleList(
    (0): TKANCell(
      (tkan_sub_layers): ModuleList(
        (0): KANLinear(
          (linear): Linear(in_features=150, out_features=150, bias=True)
          (layer_norm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
  )
)

In [7]:
model = TKAT(
            inpput_=sequence_length, 
            num_unknown_features=5, 
            num_known_features=3, 
            num_embedding=num_embedding, 
            num_hidden=num_hidden, 
            num_heads=num_heads, 
            n_ahead=n_ahead, 
            use_tkan=True
        )
model

TKAT(
  (embedding_layer): EmbeddingLayer(
    (dense_layers): ModuleList(
      (0-18): 19 x Linear(in_features=1, out_features=100, bias=True)
    )
  )
  (vsn_past_features): VariableSelectionNetwork()
  (vsn_future_features): VariableSelectionNetwork()
  (encoder): RecurrentLayer(
    (layer): TKAN(
      (tkan_cells): ModuleList(
        (0): TKANCell(
          (tkan_sub_layers): ModuleList(
            (0): KANLinear(
              (linear): Linear(in_features=100, out_features=100, bias=True)
              (layer_norm): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
            )
          )
        )
      )
    )
  )
  (decoder): RecurrentLayer(
    (layer): TKAN(
      (tkan_cells): ModuleList(
        (0): TKANCell(
          (tkan_sub_layers): ModuleList(
            (0): KANLinear(
              (linear): Linear(in_features=100, out_features=100, bias=True)
              (layer_norm): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
            )
          )
 

In [14]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params}")

criterion = nn.CrossEntropyLoss()

# LBFGS is really slow
# optimizer = optim.LBFGS(model.parameters(), lr=0.01)
# Adam works with very low lr
optimizer = optim.Adam(model.parameters(), lr=0.0002)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Total trainable parameters: 158950


In [15]:

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        print(f"data: {data.shape}")

        if isinstance(optimizer, optim.LBFGS):
            def closure():
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                return loss
            loss = optimizer.step(closure)
        else:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            loss = loss.item()
            
        total_loss += loss
        
    return total_loss / len(train_loader)

def validate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return total_loss / len(test_loader), correct / len(test_loader.dataset)    

In [16]:
train_losses = []
test_losses = []

epochs = 30
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = validate(model, test_loader, criterion, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, '
        f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}')

data: torch.Size([1, 150, 19])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (150x19 and 300x150)

Search Stack Overflow
